## Infos ( à supprimer ensuite)

## Variables à ajouter issues des recherches biblio ( à supprimer ensuite)

Variable(s): 

- Population des pays représentés 
- Variable indiquant si le pays est communiste ou pas 
- Variable indiquant si le pays est stable ou pas 
- Variable indiquant si le pays est émergeant ou pas ( otherwise pauvre , riche à peaufiner par la suite) 
- Variable indiquant si le pays a la culture du sport ou pas. 
- Variable indiquant le PIB par habitant 
- Variable indiquant les financements alloués aux sports (olympiques ou pas)


Il faut créer le train et le test avec des données 2024 , il faut les construire , il faut aussi savoir traiter le fait que cette fois ci la Russie ne sera pas disponible .
Faire indice : pays perturbé 1 avec Russie par exemple ou bien on enlève la Russie  (contexte géopolitique simple, on tah les années où j'ai eu un pb )

But c'est de prédire le nombre de médailles pour chaque pays , indicateur simple 
EXTRA POUR AVOIR 20 : Attribuer un poid aux médailles -> pour potentiellement savoir le nb d'or , argent , bronze

Attention au COVID 19 , influence public , web scrapping sur 30 ans de tous les pays qui ont organisé les jeux 
Attention aux pays pas autorisés à concourir 
Pays en guerre qui envoient moins d'athlètes 
Ukraine 
Palestine 
Iles caiman -> doivent être rattachées à un pays 

JO hiver 
JO été Corrélation 
Apparition des femmes dans les JO ( échantillonnage ) 

Culture du sport : Comment la créer ? 
Budget alloué aux sports (investissement , infrastructures ... ) 
Nb de professionnels 
Nb de licencés 
Nb de sports olympiques disponibles à l'école 

Variable : Pays Communiste ou pas 

Croissance pays économique 

Athlète à partir de 25 ans il n'a plus le niveau : on commence à partir de 2000 , JO Sydney 

Indice dvp des pays à recevoir 

Variable pays organisateur 

Sport qui ont des gens avec un âge bien définie (exemple skate Japonais c'est que des jeunes <= 18 ans)

Dataset coupes mondes de discipline  
    
Variable Note selon l'âge 

Athlète qui sont comptés comme participant à deux pays 

Variable : religion majoritaire pays 

Espérance de vie par pays 
Variable pays dvp ou pas 


Recupérer données JO 2018 et 2021

Variable Poids par Médailles : échelloner les JO de 1 à 10 pour 
Variable Poids par année : 

# Introduction  (Business Understanding) 

## Détermination objectifs métier , analytique du projet 


Objectifs métiers du projet :

Promotion de la performance sportive : Utiliser les données des Jeux olympiques pour améliorer la performance sportive.

Optimisation des ressources : Identifier les sports les plus prometteurs pour optimiser les investissements sportifs.

Prédiction des tendances : Prédire les performances futures des pays et des athlètes en se basant sur les données historiques.

Analyse de la participation : Analyser la participation mondiale aux Jeux olympiques pour identifier les opportunités de croissance.

-------
Objectifs analytiques du projet :

Analyse des performances : Évaluer les performances des pays, des équipes et des athlètes aux Jeux olympiques.

Identification des facteurs de réussite : Identifier les clés du succès dans différents sports et disciplines.

Visualisation des données : Créer des visualisations interactives pour présenter les tendances et les comparaisons de manière compréhensible.

Modélisation prédictive : Développer des modèles prédictifs pour estimer les performances futures.

## Identifier les contraintes , limites et risque du projet

Qualité des données : Les données historiques peuvent être incomplètes ou imprécises.

Variabilité des performances : Les performances sportives sont influencées par de nombreux facteurs difficiles à quantifier.

Interprétation des résultats : Les résultats doivent être interprétés avec prudence en raison de la subjectivité des performances sportives.

Protection des données : Respecter les règles de confidentialité et de protection des données lors de l'analyse des données.

Dépendance aux technologies : Le projet dépend de technologies telles que l'analyse de données, la modélisation statistique, etc.

## Chargement des packages si besoin 

In [1]:
%load_ext ipython_memory_usage 
#!pip install ydata_profiling

#from ydata_profiling import ProfileReport

import pandas as pd
import numpy as np
import ipython_memory_usage.ipython_memory_usage as imu
from describe_csv import *  # charge le fichier describe_csv.py par chunk de 10 000 (10**4)

import math
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import scipy as sp
from scipy.stats import chi2_contingency
from pivottablejs import pivot_ui
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn.pipeline import make_pipeline, make_union
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from itertools import combinations

from bs4 import BeautifulSoup
import requests
import re
import datetime

imu.start_watching_memory() 

Enabling IPython Memory Usage, use %imu_start to begin, %imu_stop to end
In [1] used 123.6 MiB RAM in 13.53s (system mean cpu 0%, single max cpu 0%), peaked 0.0 MiB above final usage, current RAM usage now 173.3 MiB


# CODE  

## Bases de données disponibles 

In [2]:
df_athlete=pd.read_csv('data/athlete_events.csv')
df_region=pd.read_csv('data/noc_regions.csv')
df_dictionnary=pd.read_csv('data/worldcities.csv', sep=';')

In [2] used 30.1 MiB RAM in 5.65s (system mean cpu 21%, single max cpu 100%), peaked 117.8 MiB above final usage, current RAM usage now 203.4 MiB


#### Pays organisateurs

In [3]:
# TODO: Changer le nom des variables
url_host_og='https://www.topendsports.com/events/summer/hosts/list.htm'
response=requests.get(url_host_og)
url_host_citi='https://en.wikipedia.org/wiki/List_of_Olympic_Games_host_cities'

response_2=requests.get(url_host_citi)

soup_2=BeautifulSoup(response_2.text,'html.parser')

In [3] used 43.0 MiB RAM in 2.17s (system mean cpu 24%, single max cpu 100%), peaked 0.0 MiB above final usage, current RAM usage now 246.3 MiB


In [67]:
# TODO: Changer le nom des variables

# Récupération des éléments de la page
L_pays_1896=[]
L_pays_1948=[]
L_pays_2000=[]
L_pays=[]

elements=soup_2.findAll('span',class_='datasortkey')

for element in elements:
    pays=element.find('a').text
    L_pays.append(pays)
L_pays_1896=L_pays[:7]
L_pays_1948=L_pays[14:42]
L_pays_2000=L_pays[42:58]

In [67] used -2.7 MiB RAM in 0.12s (system mean cpu 22%, single max cpu 34%), peaked 0.0 MiB above final usage, current RAM usage now 21.8 MiB


In [68]:
L_pays_1948

['Switzerland',
 'United Kingdom',
 'Norway',
 'Finland',
 'Italy',
 'Australia',
 'Sweden',
 'United States',
 'Italy',
 'Austria',
 'Japan',
 'France',
 'Mexico',
 'Japan',
 'West Germany',
 'Austria',
 'Canada',
 'United States',
 'Soviet Union',
 'Yugoslavia',
 'United States',
 'Canada',
 'South Korea',
 'France',
 'Spain',
 'Norway',
 'United States',
 'Japan']

In [68] used 2.4 MiB RAM in 0.11s (system mean cpu 21%, single max cpu 32%), peaked 0.0 MiB above final usage, current RAM usage now 24.2 MiB


In [69]:
L_pays_1948.pop(6)
L_pays_1948.pop(-3)

'Norway'

In [69] used -7.1 MiB RAM in 0.11s (system mean cpu 15%, single max cpu 22%), peaked 0.0 MiB above final usage, current RAM usage now 17.2 MiB


In [71]:
L_pays_1948

['Switzerland',
 'United Kingdom',
 'Norway',
 'Finland',
 'Italy',
 'Australia',
 'United States',
 'Italy',
 'Austria',
 'Japan',
 'France',
 'Mexico',
 'Japan',
 'West Germany',
 'Austria',
 'Canada',
 'United States',
 'Soviet Union',
 'Yugoslavia',
 'United States',
 'Canada',
 'South Korea',
 'France',
 'Spain',
 'United States',
 'Japan']

In [71] used 0.2 MiB RAM in 0.11s (system mean cpu 16%, single max cpu 25%), peaked 0.0 MiB above final usage, current RAM usage now 39.9 MiB


## Pays host des  années à partir de 2000 

In [16]:
L_annees_1896=[1896 + i for i in range(0,28,4)]
L_annees_1896

[1896, 1900, 1904, 1908, 1912, 1916, 1920]

In [16] used 0.6 MiB RAM in 0.11s (system mean cpu 14%, single max cpu 22%), peaked 0.0 MiB above final usage, current RAM usage now 17.5 MiB


In [52]:
L_annees_1948=[1948+ i for i in range(0,52,2)]
L_annees_1948

26

In [52] used -3.7 MiB RAM in 0.11s (system mean cpu 15%, single max cpu 25%), peaked 0.0 MiB above final usage, current RAM usage now 19.8 MiB


In [74]:
# Itération des années
L_annees_2000=[2000 + i  for i in range(0,34,2)]
L_annees_2000.pop(-2)

L_annees_2000

[2000,
 2002,
 2004,
 2006,
 2008,
 2010,
 2012,
 2014,
 2016,
 2018,
 2020,
 2022,
 2024,
 2026,
 2028,
 2032]

In [74] used -21.4 MiB RAM in 0.11s (system mean cpu 30%, single max cpu 48%), peaked 0.0 MiB above final usage, current RAM usage now 17.4 MiB


In [76]:
# On lie les pays et les années

L_pays_annee=[]
for i in range(len(L_pays_1948)):
    L_pays_annee.append((L_pays_1948[i],L_annees_1948[i]))

    
for i in range(len(L_pays_2000)):
    L_pays_annee.append((L_pays_2000[i],L_annees_2000[i]))

df_hote=pd.DataFrame(L_pays_annee,columns=['Host_country','Year'])

df_hote
#L_pays_annee

,Host_country,Year
0,Switzerland,1948
1,United Kingdom,1950
2,Norway,1952
3,Finland,1954
4,Italy,1956
5,Australia,1958
6,United States,1960
7,Italy,1962
8,Austria,1964
9,Japan,1966


In [76] used 9.6 MiB RAM in 0.20s (system mean cpu 41%, single max cpu 63%), peaked 0.0 MiB above final usage, current RAM usage now 28.3 MiB


#### Pays avec un passé communiste

In [ ]:
url_soviet='https://en.wikipedia.org/wiki/Post-Soviet_states'
response_soviet=requests.get(url_soviet)
soup_soviet=BeautifulSoup(response_soviet.text,'html.parser')

L_pays_soviet=[]

pays_soviet=soup_soviet.findAll('b')
#print(pays_soviet)
for pays in pays_soviet:
    b=pays.find('a')
    if b is not None:
        L_pays_soviet.append(b.text)
L_pays_soviet=L_pays_soviet[:15]

complete_urss_regime=lambda x : 1 if x in L_pays_soviet else 0

#### Scrap population et GDP

In [ ]:
url_pop_gpa='https://www.worldometers.info/gdp/gdp-by-country/'
response_pop_gpa=requests.get(url_pop_gpa)
soup_pop_gpa=BeautifulSoup(response_pop_gpa.text,'html.parser')

L_pays=[]

all_countries=soup_pop_gpa.findAll('td')

for country in all_countries:
    a=country.find('a')
    if a is not None:
        L_pays.append(a.text)
len(L_pays)

In [ ]:
all_info=soup_pop_gpa.findAll('td',attrs={'style':'font-weight: bold; text-align:right;'})

numbers=[]
for i,info in enumerate(all_info):
    b=re.findall(r'\d+\b', info.text)
    b=''.join(b)
    numbers.append(b)
   
population=[]
gpd_per_habitant=[]
PIB_liste=[]
for i in range(0,len(numbers),6):
    PIB_liste.append(int(numbers[i]))
for i in range(3,len(numbers),6):
    population.append(int(numbers[i]))
for i in range(4,len(numbers),6):
    gpd_per_habitant.append(int(numbers[i]))

print(len(PIB_liste))
PIB_liste

In [ ]:
# On replace les pays qui n'ont pas le même nom
df_pop_gpd=pd.DataFrame({'Country':L_pays,'PIB':PIB_liste,'Population':population,'PIB/habitant':gpd_per_habitant})

df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Trinidad and Tobago','Trinidad')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('State of Palestine','Palestine, Occupied Territories')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Singapore','Singapour')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace('Czech Republic (Czechia)','Czech Republic')
df_pop_gpd['Country']=df_pop_gpd['Country'].replace("Côte d'Ivoire" ,"Cote d'Ivoire")

df_pop_gpd

In [ ]:
# Pays qui manquaient sur le site scrappé, données récupérées manuellement
df_pop_gpd=pd.concat([df_pop_gpd,pd.DataFrame([['Taiwan',762670000000,23923276,44460],
                                               ['Cook Islands',220000000,17044,13558],
                                               ['Eritrea',1960000000,3684000,1067],
                                               ['Kosovo',9410000000,1760000,5340],
                                               ['Lebanon',23130000000,5425542,4136],
                                               ['Netherlands Antilles',3810000000,27148,11400],
                                               ['South Sudan',12000000000,11000000,467], 
                                               ['American Samoa',871000000,43914,19673],
                                              ['Aruba',3545000000,106445,333000],
                                              ['Bermuda',7645000000,63532,118774],
                                               ['British Virgin Islands',1751000000,31707,34246],
                                               ['Virgin Islands',4440000000,105413,41974],
                                               ['Cayman Islands',6845000000,69310,99624],
                                               ['Cuba',633400000000,11179471,56495],
                                               ['Guam',6123000000,173825,35904],
                                               ['North Korea',48000000000,26070000,33719],
                                               ['Liechtenstein',7710000000,39039,197504],
                                               ['Monaco',8784000000,39050,240862],
                                               ['Puerto Rico',113400000000,3260314,35208],
                                               ['San Marino',1855000000,33642,53000],
                                               ['Syria',8970000000,22130000,420],
                                               ['Venezuela',92000000000,29254044,2624],
                                               ['Virgin Islands*',4444000000,105413,41976],
                                               ['Afghanistan',14583135237,42239000,363.73],
                                               ['Bhutan',2539551327,782455,3560],
                                               ['Brunei',16760000000,449000,37152],
                                               ['Burma',62260000000,54180000,1149],
                                               ['Cape Verde',2227000000,593000,3754],
                                               ['Djibouti',3515000000,1121000,3136],
                                               ["East Timor (Timor-Leste)",3205000000,1341000,2389],
                                               ['Macedonia',13560000000,2058000,6591],
                                               ['Nauru',151600000,12668,11970],
                                               ['Palau',232000000,18055,12921],
                                               ['Saint Kitts and Nevis',965000000,47000,20262],
                                               ['Saint Vincent and the Grenadines',948000000,9123,9125],
                                               ['Sao Tome and Principe',542000000,227380,2386],
                                               ['Somalia',10042000000,17000000,592],
                                               ['Swaziland',4791000000,1202000,3986],
                                               ['Tonga',469000000,106858,4425],
                                               ['Turkmenistan',56540000000,6431000,8792],
                                               ['Yemen',21160000000,33700000,650]
                                               ],
                                             columns=['Country','PIB','Population','PIB/habitant'])],ignore_index=True)

In [ ]:
## Dictionnaire pour remplir les valeurs NaN de Pop et PIB sans changer les valeurs de pop et PIB per habitant précédentes
dictio_pop_gpd=dict(zip(df_pop_gpd['Country'],df_pop_gpd['PIB']))

dictio_pop_gpd

#### JO de Tokyo

Pour les jeux de 2020 (été, Tokyo), nous avons trouvé le dataset suivant : https://www.kaggle.com/datasets/piterfm/tokyo-2020-olympics. Après téléchargement des différents datasets, nous le plaçons dans `./data/additionnal/tokyo2020`

In [ ]:
# Chargement des données
df_tokyo_athletes = pd.read_csv('data/additionnal/tokyo2020/athletes.csv')
df_tokyo_medals_total = pd.read_csv('data/additionnal/tokyo2020/medals_total.csv')
df_tokyo_medals = pd.read_csv('data/additionnal/tokyo2020/medals.csv')

In [ ]:
# Création d'un dataframe avec les même colonnes que df_athlete
df_complete_tokyo2020 = pd.DataFrame(columns=df_athlete.columns)

# On s'assure qu'il n'y a pas de doublons
df_tokyo_athletes.drop_duplicates(inplace=True)

df_merged_bis = pd.merge(df_tokyo_athletes, df_tokyo_medals, how='left', left_on=['name', 'discipline_code'], right_on=['athlete_name', 'discipline_code'])

In [ ]:
athletes_to_be_added = []
# Itération dans df_tokyo_athletes pour ajouter dans athletes_to_be_added, qui sera ensuite ajouté au dataframe df_complete_tokyo2020

for index, row in df_tokyo_athletes.iterrows():
    athlete_name = row['name']
    athlete_sex = str(row['gender'])[0] if type(row['gender'] != float) else np.nan
    athlete_age = (2021 - datetime.date.fromisoformat(str(row['birth_date'])).year) if (type(row['birth_date'] == str) and str(row['birth_date']).lower() != 'nan' ) else np.nan
    athlete_height_in_m = (int(float(row['height_m/ft'].split('/')[0]) *100)) if (type(row['height_m/ft']) != float) else np.nan
    athlete_team = row['country']
    athlete_noc = row['country_code']
    athlete_games = 'Tokyo 2020'
    athlete_year = 2020
    athlete_season = 'Summer'
    athlete_city = 'Tokyo'
    athlete_discipline = row['discipline']
    athletes_to_be_added.append({
        'Name': athlete_name,
        'Sex': athlete_sex, 
        'Age': athlete_age,
        'Height': athlete_height_in_m,
        'Team': athlete_team,
        'NOC': athlete_noc,
        'Games': athlete_games,
        'Year': athlete_year,
        'Season': athlete_season,
        'City': athlete_city,
        'Sport': athlete_discipline,
    })

df_complete_tokyo2020 = df_complete_tokyo2020.from_records(athletes_to_be_added)

In [ ]:
# On effectue une jointure droite entre les deux dataframes, de manière à conserver seulement les athlètes avec différentes médailles dans différentes disciplines
df_merged = pd.merge(df_complete_tokyo2020, df_tokyo_medals, left_on=['Name', 'Sport'], right_on=['athlete_name', 'discipline'], how='left')
df_merged.drop(columns=['medal_code', 'medal_date', 'athlete_short_name',
       'athlete_name', 'athlete_sex', 'athlete_link', 'country_code',
       'discipline_code', 'country', 'discipline'], inplace=True)

df_merged.drop_duplicates(inplace=True)

df_merged['medal_type'] = df_merged['medal_type'].apply(lambda x: x.replace('Medal', '').strip() if type(x) != float else x)
df_merged.rename(columns={'medal_type': 'Medal', 'event': 'Event'}, inplace=True)

In [ ]:
# On récupère le nom des médaillés seulement, pour les retirer du dataframe final (qui ne contient pas encore les médailles obtenues par les athlètes), avant de fusionner
# le dataframe des médaillés seulement et celui des athlètes sans médailles
medailles_seulement = df_merged.Name.unique()
df_complete_tokyo2020.drop(df_complete_tokyo2020[df_complete_tokyo2020['Name'].isin(medailles_seulement)].index, inplace=True)
df_complete_tokyo2020 = pd.concat([df_complete_tokyo2020, df_merged])
df_complete_tokyo2020

In [ ]:
# On remplace les valeurs 'n' en NaN pour le sexe
df_complete_tokyo2020['Sex'] = df_complete_tokyo2020['Sex'].apply(lambda x: np.NAN if x == 'n' else x)

In [ ]:
# On merge avec df_athlete
df_athlete = pd.concat([df_athlete,df_complete_tokyo2020 ])

## Optimisation des datasets
   Dans un souci d'optimisation de l'espace mémoire occupé, nous allons faire en sorte d'optimiser les différents dataframe que nous utilisons. 
Pour mener à bien ceci, nous allons des fonctions contenues dans `describe_csv.py`, en les améliorant pour automatiser le processus.

In [ ]:
#### Code Nathan à ajouter
# Jusqu'à maintenant, nous utilisons le dataframe df_athlete. Nous allons définir différentes fonctions pour optimiser les types et ainsi optimiser l'espace mémoire occupé

from pandas.api.types import is_numeric_dtype


def define_and_use_optimal_type(df:pd.DataFrame):
    for column in df.columns:
        if(is_numeric_dtype(df[column]) == True):
            # Set optimal type else do nothing
            optimalype = optimal_type(df[column], df[column].min(), df[column].max())
            df[column] = df[column].astype(optimalype)
    return df

## Nettoyage

In [ ]:
## Ajouter le code FLO

### Dataframe dictionnary

In [ ]:
df_dictionnary=pd.read_csv('data/dictionary.csv')

df_dictionnary=df_dictionnary.rename(columns={'GDP per Capita':'PIB/habitant','Code':'NOC'})

df_dictionnary.isna().sum()

In [ ]:
df_dictionnary['Country']=df_dictionnary['Country'].replace('Puerto Rico*','Puerto Rico')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Hong Kong*','Hong Kong')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Korea, South','South Korea')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Korea, North','North Korea')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Liban','Lebanon')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Congo, Dem Rep','DR Congo')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Cayman Islands*','Cayman Islands')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Bermuda*','Bermuda')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Virgin Islands*','Virgin Islands')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Aruba*','Aruba')
df_dictionnary['Country']=df_dictionnary['Country'].replace('American Samoa*','American Samoa')
df_dictionnary['Country']=df_dictionnary['Country'].replace('Netherlands Antilles*','Netherlands Antilles')

### Dataframe région

In [ ]:
df_region=df_region.rename(columns={'region':'Country'})

Ajout des îles Tuvalu

In [ ]:
df_region[df_region['Country'].isnull()]

In [ ]:
df_region=df_region.query(" NOC != 'ROT' and NOC != 'UNK' ") # Penser à ajouter Singapour

df_region['Country']=df_region['Country'].fillna('Tuvalu')

In [ ]:
df_region

## Merge des dataframes et traitement des valeurs nulles

### Opérations sur les pays

In [ ]:
df_complete=pd.merge(pd.merge(df_athlete,df_region,on='NOC',how='left'),df_dictionnary,on='NOC',how='left')

# On prend seulement les JO d'été
df_complete=df_complete[df_complete['Season']=='Summer']
df_complete=df_complete[df_complete['Year'] >= 2000]

In [ ]:
# Attribution des pays en fonction des NOC 
df_complete=pd.merge(df_complete,df_region,on='NOC',how='left')
df_complete['Country']=df_complete['Country_y'].fillna(df_complete['Country_x'])
df_complete= df_complete.drop(['Country_x', 'Country_y', 'notes_x','notes_y'], axis=1)

In [ ]:
# On retire les NOC (réfugiés)
df_complete=df_complete.query(" NOC !='ROC' ")

In [ ]:
# Analyse des valeurs nulles 
df_complete.isna().sum()

# 156 sports où on sait pas ce que c'est .... et des country (26 221) qui ne sont pas attribués non plus 

In [ ]:
# Attribution des 
liste_pays=df_region['Country'].tolist()
liste_NOC=df_region['NOC'].tolist()

dictio_NOC_pays=dict(zip(liste_NOC,liste_pays))

dictio_NOC_pays['SGP']='Singapour'
dictio_NOC_pays['LBN']='Liban'

df_complete['Country']=df_complete['Country'].fillna(df_complete['NOC'].map(dictio_NOC_pays))


In [ ]:
# Il y a quatre NOC qui ne correspondent à aucun pays dans notre EOR : réfugiés , ROT réfugiés aussi , LBN c'est le Liban (Lebanon en anglais )
df_complete=df_complete.query(" NOC !='ROT' and NOC !='EOR' ")

In [ ]:
# On ajoute les pays organisateurs pour chaque ligne scrappée précédemment
df_complete=pd.merge(df_complete,df_hote,on='Year',how='left')

In [ ]:
# On ajoute une colonne booléenne (l'athlète participe aux jeux dans son propre pays)
compare_col=lambda x : 1 if x['Country']==x['Host_country'] else 0 

df_complete['Participation_own_country']=df_complete.apply(compare_col,axis=1)
df_complete[df_complete['Participation_own_country']==1]

In [ ]:
# On récupère les pays communistes scrappés précédemment
df_complete['Soviet_past']=df_complete['Country'].apply(complete_urss_regime)

In [ ]:
# On renomme les pays pour pouvoir merge avec les données scrappées 
# Changement de noms pour les pays

df_complete['Country']=df_complete['Country'].replace('Puerto Rico*','Puerto Rico')
df_complete['Country']=df_complete['Country'].replace('Hong Kong*','Hong Kong')
df_complete['Country']=df_complete['Country'].replace('Korea, South','South Korea')
df_complete['Country']=df_complete['Country'].replace('Korea, North','North Korea')
df_complete['Country']=df_complete['Country'].replace('Liban','Lebanon')
df_complete['Country']=df_complete['Country'].replace('Congo, Dem Rep','DR Congo')
df_complete['Country']=df_complete['Country'].replace('Cayman Islands*','Cayman Islands')
df_complete['Country']=df_complete['Country'].replace('Bermuda*','Bermuda')
df_complete['Country']=df_complete['Country'].replace('Virgin Islands*','Virgin Islands')
df_complete['Country']=df_complete['Country'].replace('Aruba*','Aruba')
df_complete['Country']=df_complete['Country'].replace('American Samoa*','American Samoa')
df_complete['Country']=df_complete['Country'].replace('Netherlands Antilles*','Netherlands Antilles')

# URSS 
df_complete.loc[(df_complete['Team'] == 'Soviet Union') & (df_complete['NOC'] == 'URS'), ['Team', 'NOC']] = ['Russia', 'RUS'] # URSS période 1952-1988

# Réfugiés
df_complete.loc[(df_complete['Team'] == 'ROC') & (df_complete['NOC'] == 'ROC'), ['Team', 'NOC']] = ['Germany', 'RUS'] # JO 2020

# GERMANY
df_complete.loc[(df_complete['Team'] == 'East Germany') & (df_complete['NOC'] == 'GDR'), ['Team', 'NOC']] = ['Germany', 'GER']

# CHINA
df_complete.loc[df_complete['Team'] == "People's Republic of China", ['Team', 'NOC']] = ['China', 'CHN']

# UNITED STATES
df_complete.loc[df_complete['Team'] == "United States of America", ['Team', 'NOC']] = ['United States', 'USA']

In [ ]:
# Normalisation des pays. Certains sont écrits avec des suffixes (Ex: Brazil-1, Brazil-2)
replacement_dict = {}

old_countries_to_process = df_complete.Team.unique()

for i in old_countries_to_process:
    replacement_dict[i] = re.sub(r'[\d-]', '', i)


final_df = df_complete.replace({"Team": replacement_dict})

In [ ]:
df_test_na_pop=df_complete[df_complete['Population'].isna()]

# On merge les données de df_test_na_pop et df_complete
df_test_na_pop=pd.merge(df_test_na_pop,df_pop_gpd,on='Country',how='left')
df_test_na_pop.isna().sum()

In [ ]:
# On nettoie les colonnes résiduelles
df_test_na_pop=df_test_na_pop.drop(['Population_x'],axis=1)
df_test_na_pop=df_test_na_pop.drop(['PIB/habitant_x'],axis=1)

# On renomme les colonnes
df_test_na_pop=df_test_na_pop.rename(columns={'Population_y':'Population'})
df_test_na_pop=df_test_na_pop.rename(columns={'PIB/habitant_y':'PIB/habitant'})

# On nettoie les données nulles dans la colonne Population
df_test_na_pop=df_test_na_pop.dropna(subset='Population')
df_complete=df_complete.dropna(subset='Population')

df_complete=pd.concat([df_complete,df_test_na_pop],ignore_index=True)

### Opérations sur le PIB

In [ ]:
# Traitement des données nulles
df_test_na_pib=df_complete[df_complete['PIB/habitant'].isna()]
df_complete=df_complete.dropna(subset='PIB/habitant')

# On merge avec les données scrappées
df_test_na_pib=pd.merge(df_test_na_pib,df_pop_gpd,on='Country',how='left')

# On drop les données résiduelles
df_test_na_pib=df_test_na_pib.drop(['Population_x'],axis=1)
df_test_na_pib=df_test_na_pib.drop(['PIB/habitant_x'],axis=1)
df_test_na_pib=df_test_na_pib.drop(['PIB_x'],axis=1)

# On renomme les colonnes merge
df_test_na_pib=df_test_na_pib.rename(columns={'Population_y':'Population'})
df_test_na_pib=df_test_na_pib.rename(columns={'PIB/habitant_y':'PIB/habitant'})
df_test_na_pib=df_test_na_pib.rename(columns={'PIB_y':'PIB'})

df_complete=pd.concat([df_complete,df_test_na_pib],ignore_index=True)

In [ ]:
# Attribution du PIB aux pays manquants
df_complete['PIB']=df_complete['PIB'].fillna(df_complete['Country'].map(dictio_pop_gpd))

### Opérations sur les âges

In [ ]:
# On attribue les âges manquants en faisant la moyenne des âges par sport
table_sport_age=df_complete.groupby('Sport')['Age'].mean().apply(math.ceil).reset_index()

liste_sport=table_sport_age['Sport'].tolist()
liste_age=table_sport_age['Age'].tolist()

dictio_age_sport=dict(zip(liste_sport,liste_age))

df_complete['Age']=df_complete['Age'].fillna(df_complete['Sport'].map(dictio_age_sport))

In [ ]:
# Il y a 156 personnes qui manquent car nous n'avons pas les sports 
df_complete=df_complete.dropna(subset='Age')

### Opérations sur le sexe, le poids et les tailles

In [ ]:
df_male = df_complete[df_complete['Sex'] == 'M']
df_female = df_complete[df_complete['Sex'] == 'F']
columns_to_impute = ['Age', 'Height', 'Weight']

In [ ]:
# Créer un imputer KNN avec un nombre de voisins à considérer (k)
imputer = KNNImputer(n_neighbors=5)

# Imputer les valeurs manquantes pour les hommes
df_male[columns_to_impute] = imputer.fit_transform(df_male[columns_to_impute])

# Imputer les valeurs manquantes pour les femmes
df_female[columns_to_impute] = imputer.fit_transform(df_female[columns_to_impute])

# Fusionner les DataFrames pour obtenir le DataFrame complet
df_complete = pd.concat([df_male, df_female])

# Data understanding

## EDA (graph, pivot table ,Matrix confusion...)

In [ ]:
# TODO: Guillaume / Noé / Flo

Utilisation pandas_profiling

In [ ]:
# profile = ProfileReport(df, title='Pandas Profiling Report')
# profile.to_file("report-profiling-test.html")

EDA via pivottablejs

In [ ]:
# pivot_ui(df)

Prétraitement des données et création de variables catégorielles

In [ ]:
# # TRAITER AGE NULL (moyenne des ages)
# df_complete['Age'] = df_complete['Age'].fillna(df_complete['Age'].mean())

# # TRAITER HEIGHT NULL (moyenne des heights)
# df_complete['Height'] = df_complete['Height'].fillna(df_complete.groupby('Sex')['Height'].transform('mean'))

# # TRAITER WEIGHT NULL (moyenne des weights)
# df_complete['Weight'] = df_complete['Weight'].fillna(df_complete.groupby('Sex')['Weight'].transform('mean'))

# TRAITER SEASON NULL (0 si null, 1 sinon)
# df_complete['Season'] = np.where(df_complete['Season'].isnull(), 0, 1)

# TRAITER AGE GROUP
df_complete['AgeGroup'] = pd.cut(df_complete.Age, bins=range(0, 81, 10), right=True)

# TRAITER WEIGHT GROUP
df_complete['WeightGroup'] = pd.cut(df_complete.Weight, bins=range(0, 215, 20), right=True)

# TRAITER HEIGHT GROUP
df_complete['HeightGroup'] = pd.cut(df_complete.Height, bins=range(100, 227, 20), right=True)


In [ ]:
attrs = ['Sex', 'AgeGroup', 'HeightGroup', 'WeightGroup'] # attrs permet de sélectionner les colonnes à afficher

In [ ]:
# Paramètres de style de Seaborn
sns.set(style="whitegrid")

# Créer la figure
figure = plt.figure(figsize=(14, 10))
figure.suptitle('Répartition des médailles dans les Jeux Olympiques', fontsize=16)

# Ajuster l'espacement entre les sous-graphiques
plt.subplots_adjust(top=0.9, bottom=0.1, hspace=0.5, wspace=0.4)

# Boucle sur les attributs pour créer les sous-graphiques
for i, col in enumerate(attrs):
    plt.subplot(2, 4, i+1)
    plt.xticks(rotation=40, ha='right')  # Rotation des étiquettes sur l'axe des x
    sns.countplot(x=col, hue='Medal', data=df_complete, palette='pastel')  # Utiliser une palette de couleurs pastel
    plt.title('Répartition des médailles par ' + col)  # Titre du sous-graphique

# Améliorer la disposition des sous-graphiques
plt.tight_layout()
plt.show()

#### Analyse visuelle

Nombre d'événements dans chaque sport au fil des années 

In [ ]:
sport_year = df_complete.pivot_table(index='Sport', columns='Year', values='Event', aggfunc=lambda x: len(x.unique()))
sport_year.fillna(0, inplace=True)
sport_year = sport_year.reindex(sport_year.sum(axis=1).sort_values(ascending=False).index)

# Affichage de la heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(sport_year, annot=True, linewidths=0.05, cmap="RdYlGn")
plt.xlabel('Year', size=14, color="green")
plt.ylabel('Sport', size=14, color="green")
plt.title('Nombre d\'événements dans chaque sport au fil des années', size=18, color="green")
plt.show()

Nombre total de médailles remportées par pays et type de médaille

In [ ]:
# Filter the data to include only gold medal winners
gold_medals = df_complete[df_complete['Medal'] == 'Gold']

# Group the data by year and count the number of gold medals won each year for each country
gold_medals_by_year_and_country = gold_medals.groupby(['Year', 'Team'])['Medal'].count().unstack()

# Select the top 10 countries with the most gold medals overall
top_10_countries = gold_medals.groupby('Team')['Medal'].count().nlargest(10).index

# Plotting the curve for the evolution of gold medals over time for the top 10 countries
plt.figure(figsize=(10, 6))  # Taille de la figure

for country in top_10_countries:
    plt.plot(gold_medals_by_year_and_country.index, gold_medals_by_year_and_country[country], 
             marker='o', markersize=6, linewidth=2, label=country)  # Courbe pour chaque pays avec la légende

plt.title('Evolution des médailles en or gagnées au fil du temps (Top 10 des pays)', fontsize=16)  # Titre
plt.xlabel('Year', fontsize=14)  # Étiquette de l'axe x
plt.ylabel('Number of Gold Medals', fontsize=14)  # Étiquette de l'axe y
plt.xticks(fontsize=12)  # Taille de la police des étiquettes de l'axe x
plt.yticks(fontsize=12)  # Taille de la police des étiquettes de l'axe y
plt.grid(True)  # Afficher la grille
plt.legend(fontsize=10)  # Ajouter la légende
plt.tight_layout()  # Ajuster la mise en page
plt.show()


Nombre total de médailles remportées par pays et type de médaille

In [ ]:
# Group the data by country and count the total number of medals won by each country and each type of medal
medals_by_country = df_complete.groupby(['Team', 'Medal']).size().unstack(fill_value=0)

# Select the top 10 countries with the most total medals
top_10_countries = medals_by_country.sum(axis=1).nlargest(10).index

# Select the medal types
medal_types = ['Gold', 'Silver', 'Bronze']

# Set the position of each bar on the x-axis
x = np.arange(len(top_10_countries))

# Set the width of the bars
width = 0.2

# Assign colors to each medal type
colors = {'Gold': 'gold', 'Silver': 'silver', 'Bronze': 'peru'}

# Plotting the grouped bar graph
plt.figure(figsize=(12, 8))  # Taille de la figure

for i, medal in enumerate(medal_types):
    plt.bar(x - width + i * width, medals_by_country.loc[top_10_countries, medal], 
            width=width, label=medal, color=colors[medal])

plt.title('Nombre total de médailles remportées par pays et type de médaille', fontsize=16)  # Titre
plt.xlabel('Country', fontsize=14)  # Étiquette de l'axe x
plt.ylabel('Total Number of Medals', fontsize=14)  # Étiquette de l'axe y
plt.xticks(ticks=x, labels=top_10_countries, rotation=45, fontsize=12)  # Étiquettes de l'axe x
plt.yticks(fontsize=12)  # Taille de la police des étiquettes de l'axe y
plt.legend(fontsize=12)  # Ajouter la légende
plt.tight_layout()  # Ajuster la mise en page
plt.show()


Ratio de l'évolution des médailles d'or par athlète au fil du temps (Top 10 des pays)
(proportionnelement, quel pays obtient le pus de médailles)

In [ ]:
# Calculate the total number of athletes per country
total_athletes_per_country = df_complete.groupby('Team')['Name'].nunique()

# Filter the data to include only gold medal winners
gold_medals = df_complete[df_complete['Medal'] == 'Gold']

# Group the data by year and count the number of gold medals won each year for each country
gold_medals_by_year_and_country = gold_medals.groupby(['Year', 'Team'])['Medal'].count().unstack()

# Calculate the number of gold medals per athlete for each country and each year
gold_medals_per_athlete = gold_medals_by_year_and_country.div(total_athletes_per_country, axis=1)

# Select the top 10 countries with the most gold medals overall
top_10_countries = gold_medals.groupby('Team')['Medal'].count().nlargest(10).index

# Plotting the curve for the evolution of gold medals per athlete over time for the top 10 countries
plt.figure(figsize=(10, 6))  # Taille de la figure

for country in top_10_countries:
    plt.plot(gold_medals_per_athlete.index, gold_medals_per_athlete[country], 
             marker='o', markersize=6, linewidth=2, label=country)  # Courbe pour chaque pays avec la légende

plt.title('Ratio evolution des médailles en or par athlète au fil du temps (Top 10 des pays)', fontsize=16)  # Titre
plt.xlabel('Year', fontsize=14)  # Étiquette de l'axe x
plt.ylabel('Number of Gold Medals per Athlete', fontsize=14)  # Étiquette de l'axe y
plt.xticks(fontsize=12)  # Taille de la police des étiquettes de l'axe x
plt.yticks(fontsize=12)  # Taille de la police des étiquettes de l'axe y

plt.grid(True)  # Afficher la grille
plt.legend(fontsize=10)  # Ajouter la légende
plt.tight_layout()  # Ajuster la mise en page
plt.show()

Plus forte évolution des médailles remportées par équipe

In [ ]:
# Filter the data to include only Summer Olympic Games between 2010 and 2020
summer_games = df_complete[(df_complete['Season'] == 'Summer') & (df_complete['Year'] >= 2008) & (df_complete['Year'] <= 2020)]

# Group the data by team and year, and calculate the total number of medals won by each team in each year
medals_by_year = summer_games.groupby(['Team', 'Year'])['Medal'].count().unstack()

# Calculate the difference in the number of medals won by each team between the first year and the last year
medal_difference = medals_by_year.iloc[:, -1] - medals_by_year.iloc[:, 0]

# Sort the teams based on the difference in the number of medals won in descending order
sorted_teams = medal_difference.sort_values(ascending=False)

# Select the top 5 teams with the largest increase in medals won
top_5_teams = sorted_teams.index[:10]

# Create a line chart showing the number of medals won by the top 5 teams in each year
plt.figure(figsize=(12, 6))
for team in top_5_teams:
    plt.plot(medals_by_year.loc[team], label=team)
plt.title('Plus forte évolution des médailles remportées par équipe', fontsize=18, color='#003f5c')
plt.xlabel('Year', fontsize=14, fontweight='bold', color='#003f5c')
plt.ylabel('Number of Medals', fontsize=14, fontweight='bold', color='#003f5c')
plt.xticks(fontsize=12, color='#003f5c')
plt.yticks(fontsize=12, color='#003f5c')
plt.grid(axis='y', linestyle='--', alpha=0.5, color='#003f5c')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Retrait des variables devenues ici inutiles
df_complete.drop(columns=['AgeGroup', 'WeightGroup', 'HeightGroup'], inplace=True)

## Traitement des valeurs aberrantes

In [ ]:
# A faire
# TODO : Guillaume

In [ ]:
df_complete.boxplot()

In [ ]:
# On va modifier les plus hauts outliers identifiés pour Age, Height et Weight par la valeur du 99ème quantile

cols = ['Age', 'Height', 'Weight']
low = .05
high = .99 # Retourne les valeurs au 99ème quantile

# Calcul des quantiles
quant_df =  df_complete[cols].quantile([low, high])
print(quant_df)

# Remplacement des outliers par le 99ème quantile
for col in cols:
    high_val = quant_df[col].loc[high]
    df_complete[col] = np.where(df_complete[col] > high_val, high_val, df_complete[col])


In [ ]:
higher = quant_df.loc[high,'Age']
higher

def func(x):
    if x >= higher:
        return higher
    else:
        return x

df_complete['Age'] = df_complete['Age'].apply(func)

In [ ]:
higher = quant_df.loc[high,'Height']
higher

def func(x):
    if x >= higher:
        return higher
    else:
        return x

df_complete['Height'] = df_complete['Height'].apply(func)

In [ ]:
higher = quant_df.loc[high,'Weight']
higher

def func(x):
    if x >= higher:
        return higher
    else:
        return x

df_complete['Weight'] = df_complete['Weight'].apply(func)

In [ ]:
# On vérifie que pour les valeurs les plus hautes, on a bien modifié la valeur

df_complete[df_complete['Age'].ge(quant_df.loc[high,'Age'])]

df_complete[df_complete['Height'].ge(quant_df.loc[Height'])]

In [ ]:
df_complete[df_complete['Weight'].ge(quant_df.loc[high,'Weight'])]

## Création d'un nouveau dataframe aggrégeant toutes les données

In [ ]:
# Normalisation des pays. Certains sont écrits avec des suffixes (Ex: Brazil-1, Brazil-2)
replacement_dict = {}

old_countries_to_process = df_complete.Team.unique()

for i in old_countries_to_process:
    replacement_dict[i] = re.sub(r'[\d-]', '', i)

df_complete = df_complete.replace({"Team": replacement_dict})
df_complete.drop_duplicates(subset=['Event', 'Team', 'Year', 'Medal'], inplace=True)
participation_own_country_soviet = df_complete[['Year','Team', 'Participation_own_country', 'Soviet_past']].drop_duplicates()

df_complete['Medal'] = df_complete['Medal'].replace(np.nan, 0)
df_complete['Medal'] = df_complete['Medal'].apply(lambda x: 0 if x==0 else 1)

medals_per_year = df_complete.groupby(['Team', 'Year']).agg({'Medal': 'sum'}).reset_index()
concat = medals_per_year.merge(participation_own_country_soviet, how='inner', on=['Team', 'Year'])

# Vérif
concat[concat['Team'] == "France"]

## Encodage des données

Encodage des médailles

In [ ]:
# df_complete = pd.read_csv('data/testetstest.csv')

In [ ]:
# Encodage du sexe
df_complete['Sex'].replace(['F','M'],['0', '1'],inplace=True)

# transform the medal into numeric values
df_complete['Medal'] = df_complete.Medal.fillna('0')

#JO Hivers = 0; été = 1
df_complete['Season'].replace(['Winter','Summer'],['0', '1'],inplace=True)
df_complete = df_complete.drop(df_complete[df_complete['Season'] == '0'].index)

# On retire les valeurs nulles de Soviet_past
df_complete['Soviet_past'].fillna(0, inplace=True)

In [ ]:
# Encodage du sport
encoder = preprocessing.OneHotEncoder(sparse_output=False)
encoder.fit(df_complete[['Sport']])
columns_name = encoder.get_feature_names_out(['Sport'])
df_encoded = pd.DataFrame(encoder.transform(df_complete[['Sport']]))
df_encoded.columns = columns_name

# Reset index of df_encoded
df_encoded.reset_index(drop=True, inplace=True)

# # Concatenate df_complete and df_encoded
df_complete = pd.concat([df_complete, df_encoded], axis=1)

df_complete = df_complete.drop('Season', axis=1)

## Problème run matrice de corrélation 

In [ ]:
# Corrélogramme
#correlation_matrix = df_complete.corr()
#plt.figure(figsize=(10, 8))

# Plot the heatmap
#sns.heatmap(correlation_matrix, fmt=".2f", cmap='coolwarm', linewidths=0.5)

On n'observe pas de forte corrélation entre les différents attributs (NOE à compléter)

In [ ]:
# Encodage de l'âge
df_complete['Age'] = pd.cut(df_complete.Age, range(0, 80, 5), right=False)

# Encodage du poids
df_complete['Weight'] = pd.cut(df_complete.Weight, range(0, 230, 10), right=False)

# Encodage de la taille
df_complete['Height'] = pd.cut(df_complete.Height, range(0, 250, 10), right=False)

In [ ]:
# PIB Groups
# Calculer les quantiles
quantiles = [0.20, 0.40, 0.60, 0.80]

# Créer les intervalles pour les groupes en utilisant les quantiles
bins = df_complete['PIB'].quantile(quantiles)

# Ajouter les bornes extrêmes
bins = [float('-inf')] + list(bins) + [float('inf')]

# Assigner les groupes en utilisant pd.cut() avec les intervalles définis
df_complete['PIB_groups'] = pd.cut(df_complete['PIB'], bins=bins, labels=[1, 2, 3, 4, 5], right=False)

# Retrait des variables inutiles
df_complete.drop(columns=['PIB'], inplace=True)
df_complete

In [ ]:
categorical_vars = df_complete.select_dtypes(include=['object', 'category'])

p_values = {}
for var1, var2 in combinations(categorical_vars.columns, 2):
    contingency_table = pd.crosstab(categorical_vars[var1], categorical_vars[var2])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    p_values[(var1, var2)] = p


dependent_vars = {var_pair: p_value for var_pair, p_value in p_values.items() if p_value < 0.05}
dependent_vars

### Nettoyage des colonnes inutiles pour l'analyse

In [ ]:
# Drop les colonnes qui ne servent à R
# df_complete.drop(columns=['Team', 'Name', 'Games', 'City', 'Event', 'Sport', 'Host_country', 'Country'], inplace=True)

In [ ]:
df_complete

In [ ]:
# Export du dataframe
df_complete.to_csv('data/df_complete.csv', index=False)

In [ ]:
df_complete.head()

## Traitement des valeurs manquantes

Effectivement notre base de données possède de nombreuses valeurs manquantes

In [ ]:
# df.isna().sum()

In [ ]:
### Première solution : Web Scrapping (code Nathan + FLO)

## Tests de corrélations/lois stats (Shapiro-Wilk, Spearman, Pearson)

In [ ]:
# TODO: Noé

## Aggrégation des données par Team/Année/Event pour pouvoir calculer les médailles

In [ ]:
# TODO: Nathan: Essayer d'appliquer le code
# Voir plus haut

In [ ]:
## Partie Flo 

In [ ]:
df_complete=df_complete.dropna(subset='Name')

In [ ]:
df_complete

In [ ]:
df_world=pd.read_csv('data/additionnal/world-data-2023.csv',sep=',')

In [ ]:
df_world=df_world.drop('Population',axis=1)
df_world=df_world.drop('Abbreviation',axis=1)
df_world=df_world.drop('Official language',axis=1)
df_world=df_world.drop('Physicians per thousand',axis=1)
df_world=df_world.drop('Capital/Major City',axis=1)
df_world=df_world.drop('Density\n(P/Km2)',axis=1)
df_world=df_world.drop('Agricultural Land( %)',axis=1)
df_world=df_world.drop('Land Area(Km2)',axis=1)
df_world=df_world.drop('Calling Code',axis=1)
df_world=df_world.drop('Currency-Code',axis=1)
df_world=df_world.drop('CPI Change (%)',axis=1)
df_world=df_world.drop('CPI',axis=1)
df_world=df_world.drop('Forested Area (%)',axis=1)
df_world=df_world.drop('GDP',axis=1)
df_world=df_world.drop('Largest city',axis=1)
df_world=df_world.drop('Maternal mortality ratio',axis=1)
df_world=df_world.drop('Tax revenue (%)',axis=1)
df_world=df_world.drop('Total tax rate',axis=1)
df_world=df_world.drop('Latitude',axis=1)
df_world=df_world.drop('Longitude',axis=1)

In [ ]:
df_world

In [ ]:
df_complete.nunique().head(20)

In [ ]:
df_complete['NOC'].value_counts()

In [ ]:
df_world.isna().sum()

In [ ]:
#df_test=pd.merge(df_complete,df_world,on='Country',how='left')

## Code à rajouter dans le main

## On va chercher les GDP des pays de l'API wolrd Bank depuis tah l'époque

In [ ]:
#pip install wbgapi

In [ ]:
import wbgapi as wb 

In [ ]:
s=wb.search('GDP current') 

In [ ]:
gdp=wb.data.DataFrame('NY.GDP.MKTP.CD').transpose()

In [ ]:
df_gdp=pd.DataFrame(gdp)

In [ ]:
df_gdp = df_gdp.transpose()
df_gdp .columns = [col.strip('YR') for col in df_gdp.columns]
#df_gdp = df_gdp.transpose()

In [ ]:
## Faut transformer cette base de données 

In [ ]:
df_gdp=df_gdp.reset_index(names='NOC')

In [ ]:
df_gdp

In [ ]:
## à chercher fct melt 

In [ ]:
years_cols=[col for col in df_gdp.columns if col.isdigit()]

In [ ]:
df_melted_gdp = pd.melt(df_gdp, id_vars=['NOC'], value_vars=years_cols, var_name='Year', value_name='GDP')

# Convertir la colonne Year en type int
df_melted_gdp['Year'] = df_melted_gdp['Year'].astype(int)

In [ ]:
df_melted_gdp

## Merge avec base données dictionnary 

In [ ]:
df_dictionnary=df_dictionnary.drop('PIB/habitant',axis=1)
df_dictionnary=df_dictionnary.drop('Population',axis=1)

In [ ]:
df_dictionnary

### base données utile pour alimenter en data notre modèle

In [ ]:
df_hist_gdp=pd.merge(df_melted_gdp,df_dictionnary,on='NOC',how='left')

In [ ]:
df_hist_gdp

## On va chercher la population des pays dans l'API World Bank depuis tah l'époque

In [ ]:
search_pop=wb.search('population total') 

In [ ]:
print(search_pop)

In [ ]:
population=wb.data.DataFrame('SP.POP.TOTL').transpose()

In [ ]:
print(population)

In [ ]:
df_population=pd.DataFrame(population)
df_population = df_population.transpose()
df_population.columns = [col.strip('YR') for col in df_population.columns]
#df_gdp = df_gdp.transpose()


In [ ]:
df_population

In [ ]:
df_population=df_population.reset_index(names='NOC')

In [ ]:
df_population

In [ ]:
years_cols=[col for col in df_population.columns if col.isdigit()]
df_melted_population = pd.melt(df_population, id_vars=['NOC'], value_vars=years_cols, 
                               var_name='Year', value_name='Population')

# Convertir la colonne Year en type int
df_melted_population['Year'] = df_melted_population['Year'].astype(int)

In [ ]:
df_melted_population

### Base de données utile pour l'apport en data de notre modèle

In [ ]:
df_hist_pop=pd.merge(df_melted_population,df_dictionnary,on='NOC',how='left')



## À voir comment on gère les problèmes avec les merge de dataframe 

In [ ]:
#test=pd.merge(df_hist_gdp,df_hist_pop,on='Year')

#test